OSSとCRALWデータ、及び単漢字から訓練する

In [1]:
import os
with open('../version.txt','r',encoding='utf-8') as f:
    for l in f:
        ver=l.rstrip()
        break
os.environ["VER"]=ver
os.environ['DATA']='../dict/data/data.'+os.environ["VER"]
os.environ['DIR']='dataset/dataset.'+os.environ["VER"]
os.environ['LOG']='logs/logs.'+os.environ["VER"]
os.environ['MODEL']='model/model.'+os.environ["VER"]

In [2]:
!echo $VER

1.8o


In [3]:
#training

In [4]:
!python ./transformer_model.py \
  --emb_size 512 \
  --nhead 8 \
  --ffn_hid_dim 2048 \
  --batch_size 64 \
  --num_encoder_layers 8 \
  --num_decoder_layers 8 \
  --lr 0.00002 \
  --dropout 0.3 \
  --num_epochs 100 \
  --device cuda \
  --earlystop_patient 3 \
  --output_dir $MODEL \
  --tensorboard_logdir $LOG \
  --train_file $DIR/train.jsonl \
  --valid_file $DIR/valid.jsonl


[00:00:11] Pre-processing sequences       ██████████████████ 0        /        0[00:00:00] Pre-processing sequences       ██████████████████ 0        /        0/home/analysis01/src/kanjikana-model/venv/lib/python3.10/site-packages/torch/nn/modules/transformer.py:392: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(
num_epochs:100
/home/analysis01/src/kanjikana-model/venv/lib/python3.10/site-packages/torch/nn/functional.py:6041: UserWarning: Support for mismatched key_padding_mask and attn_mask is deprecated. Use same type for both instead.
  warnings.warn(
/home/analysis01/src/kanjikana-model/train/./transformer_model.py:377: UserWarning: Using a non-tuple sequence for multidimensional indexing is deprecated and will be changed in pytorch 2.9; use x[tuple(seq)] instead of x[seq]. In pytorch 2.9 this will be interpreted as tensor index

In [5]:
%load_ext tensorboard

In [6]:
%tensorboard --logdir logs/logs.$VER

In [7]:
!python ./generate.py \
    --test_file $DIR/test.jsonl \
    --model_file $MODEL/checkpoint_best.pt \
    --outfile $DIR/generate.txt \
    --source_lang kana \
    --target_lang kanji \
    --device cpu \
    --nbest 5 \
    --beam_width 5 \
    --max_len 100 \
    --search beam

Traceback (most recent call last):
  File "/home/analysis01/src/kanjikana-model/train/./generate.py", line 191, in <module>
    main()
  File "/home/analysis01/src/kanjikana-model/train/./generate.py", line 187, in main
    KanjiKanaTransformerTest(args).generate()
  File "/home/analysis01/src/kanjikana-model/train/./generate.py", line 131, in generate
    last_checkpoint = torch.load(self.args.model_file, map_location=torch.device(self.args.device))
  File "/home/analysis01/src/kanjikana-model/venv/lib/python3.10/site-packages/torch/serialization.py", line 1484, in load
    with _open_file_like(f, "rb") as opened_file:
  File "/home/analysis01/src/kanjikana-model/venv/lib/python3.10/site-packages/torch/serialization.py", line 759, in _open_file_like
    return _open_file(name_or_buffer, mode)
  File "/home/analysis01/src/kanjikana-model/venv/lib/python3.10/site-packages/torch/serialization.py", line 740, in __init__
    super().__init__(open(name, mode))
FileNotFoundError: [Errno 2] N

In [8]:
!python evaluate.py --infile $DIR/generate.txt

dataset/dataset.1.8o/generate.txt,acc=0.899642873287826


In [9]:
!date

2025年 10月  3日 金曜日 11:34:23 JST
